In [2]:
pip install mord

Note: you may need to restart the kernel to use updated packages.


In [14]:
%matplotlib inline
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score

In [24]:
imdb_df = pd.read_csv('projectData.csv')
imdb_df = imdb_df.set_index('tconst')

imdb_df = imdb_df.drop('numVotes', axis=1)
imdb_df = imdb_df.drop('writers', axis=1)

In [25]:
genres_df = imdb_df['genres'].str.get_dummies(sep=',')
genres_df = genres_df.add_prefix('genre_')
imdb_df = pd.concat([imdb_df, genres_df], axis=1)
imdb_df = imdb_df.drop('genres', axis=1)


directors_df = imdb_df['directors'].str.get_dummies(sep=',')
directors_df = directors_df.add_prefix('dir_')
imdb_df = pd.concat([imdb_df, directors_df], axis=1)
imdb_df = imdb_df.drop('directors', axis=1)


actor_df = imdb_df['actor'].str.get_dummies(sep=',')
actor_df = actor_df.add_prefix('actor_')
imdb_df = pd.concat([imdb_df, actor_df], axis=1)
imdb_df = imdb_df.drop('actor', axis=1)

In [6]:
# define the bins
bins = [0, 2.4, 5.0, 7.4, 10]

# define the bin labels
labels = ['Terrible', 'Poor', 'Average', 'Excellent']

# create a new column with the bin labels
imdb_df['binRating'] = pd.cut(imdb_df['averageRating'], bins=bins, labels=labels, include_lowest=True)

imdb_df = imdb_df.drop('averageRating', axis=1)
imdb_df = pd.get_dummies(imdb_df, columns=['binRating'], prefix='bin')
imdb_df.head()

,runtimeMinutes,genre_Action,genre_Adventure,genre_Animation,genre_Biography,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,...,actor_nm9500254,actor_nm9503133,actor_nm9576872,actor_nm9858131,actor_nm9862858,actor_nm9877392,bin_Terrible,bin_Poor,bin_Average,bin_Excellent
tconst,,,,,,,,,,,,,,,,,,,,,
tt0035423,118,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
tt0118589,104,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
tt0118694,98,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
tt0120202,105,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
tt0120630,84,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
X = imdb_df.drop(['bin_Average', 'bin_Excellent', 'bin_Poor', 'bin_Terrible'], axis=1)
y = imdb_df[['bin_Excellent']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=102)

In [10]:
from sklearn.multioutput import MultiOutputClassifier

log_reg = LogisticRegression(max_iter=1000000)
# log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

score = log_reg.score(X_test, y_test)
print(f"Accuracy: {score}")

/Users/wileybattles/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.8440607012035584


In [32]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

y_train = y_train.ravel()
y_test = y_test.ravel()

# Create a grid of hyperparameter values to search over.
C_values = np.logspace(-3, 3, 7)
penalty_values = ["l1", "l2"]

# Create a logistic regression model.
log_reg = LogisticRegression()

# Fit the model to the training data.
log_reg_cv = GridSearchCV(log_reg,
                          {"C": C_values, "penalty": penalty_values},
                          cv=10)
log_reg_cv.fit(X_train, y_train)

# Evaluate the model on the test data.
score = log_reg_cv.score(X_test, y_test)
print(f"Accuracy: {score}")

# Select the combination of hyperparameter values that results in the best model performance.
best_params = log_reg_cv.best_params_
print(f"Best hyperparameters: {best_params}")

AttributeError: 'DataFrame' object has no attribute 'ravel'